In [4]:
import pandas as pd
import numpy as np
import keras
import joblib
import sklearn
import sklearn.metrics
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import load_model
import sklearn.model_selection
from sklearn.preprocessing import StandardScaler


In [5]:
#Seperates input and output
df = pd.read_csv("symptoms.csv")

dictionary = dict(ALLERGY = 0, COLD = 1, COVID = 2, FLU = 3)
print(dictionary)

predict = 'TYPE'
X = df.drop(columns=predict)
y = df[predict]

colnames = X.columns

#Splitting up into train and test
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X, y, test_size = 0.2)

scaler = StandardScaler()
scaler.fit(X_train)



{'ALLERGY': 0, 'COLD': 1, 'COVID': 2, 'FLU': 3}


StandardScaler()

In [16]:
#Creating model
m=X_train.shape[0]
n=X_train.shape[1]

#Creating the neural network model
model = Sequential()
model.add(Dense(128, input_shape=(n,), activation='relu'))
model.add(Dense(256, activation = 'relu'))
model.add(Dense(128, input_shape=(n,), activation='relu'))
model.add(Dense(4, activation = 'sigmoid'))

#Compiling the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

#Chaning y_test into form that can be put into model
y_binary = keras.utils.to_categorical(y_train, num_classes=4, dtype='int')

#Fit the keras model on the dataset
model.fit(X_train, y_binary, epochs=100, verbose = 1)

#Saving the model
model.save('Symptoms_Keras_Model.h5')





Epoch 1/100
1112/1112 [==============================] - 10s 6ms/step - loss: 0.0654 - accuracy: 0.9292
Epoch 2/100
1112/1112 [==============================] - 7s 6ms/step - loss: 0.0508 - accuracy: 0.9315
Epoch 3/100
1112/1112 [==============================] - 7s 6ms/step - loss: 0.0496 - accuracy: 0.9335
Epoch 4/100
1112/1112 [==============================] - 7s 6ms/step - loss: 0.0493 - accuracy: 0.9330
Epoch 5/100
1112/1112 [==============================] - 7s 6ms/step - loss: 0.0490 - accuracy: 0.9331
Epoch 6/100
1112/1112 [==============================] - 7s 6ms/step - loss: 0.0488 - accuracy: 0.9316
Epoch 7/100
1112/1112 [==============================] - 7s 6ms/step - loss: 0.0488 - accuracy: 0.9334
Epoch 8/100
1112/1112 [==============================] - 7s 6ms/step - loss: 0.0486 - accuracy: 0.9324
Epoch 9/100
1112/1112 [==============================] - 7s 6ms/step - loss: 0.0487 - accuracy: 0.9340
Epoch 10/100
1112/1112 [==============================] - 7s 6ms/step - 

KeyboardInterrupt: 

In [19]:
#Making Predictions with saved model


model = load_model("Symptoms_Keras_Model.h5") #Loads model
predictions = model.predict(X_train) #Makes Predictions


#Converting predictions into 1s and 0s
for i in range(m):
    maxNum = predictions[i].max()
    for k in range(0, 4):      
        if predictions[i] [k] == maxNum:
            predictions[i] [k] = 1
        else:
            predictions[i] [k] = 0
    
#Making predictions into int
predictions = predictions.astype(int)

#Calculating accuracy with training data
accuracy = sklearn.metrics.accuracy_score(y_binary, predictions)
print('Accuracy: {0:.2f}'.format(accuracy * 100.0))
print('Classification Report:')
print(sklearn.metrics.classification_report(y_binary, predictions))
print('')

1112/1112 [==============================] - 4s 4ms/step


In [21]:
#Predictions with test data
model = load_model("Symptoms_Keras_Model.h5") #Loads model
predictions = model.predict(X_test) #Makes Predictions


m1=y_test.shape[0]
y_test=np.asarray(y_test).astype('int32')
y_binary_test = keras.utils.to_categorical(y_test, num_classes=4, dtype='int')
#Converting predictions into 1s and 0s
for i in range(m1):
    maxNum = predictions[i].max()
    for k in range(0, 4):      
        if predictions[i] [k] == maxNum:
            predictions[i] [k] = 1
        else:
            predictions[i] [k] = 0
    
#Making predictions into int
predictions = predictions.astype(int)

#Accuracy of predictions
accuracy = sklearn.metrics.accuracy_score(y_binary_test, predictions)
print('Accuracy: {0:.2f}'.format(accuracy * 100.0))
print('Classification Report:')
print(sklearn.metrics.classification_report(y_binary_test, predictions))
print('')

278/278 [==============================] - 1s 4ms/step
Accuracy: 90.65
Classification Report:
              precision    recall  f1-score   support

           0       0.98      0.97      0.98      3237
           1       0.37      0.49      0.42       199
           2       0.32      0.30      0.31       405
           3       0.93      0.93      0.93      5050

   micro avg       0.91      0.91      0.91      8891
   macro avg       0.65      0.67      0.66      8891
weighted avg       0.91      0.91      0.91      8891
 samples avg       0.91      0.91      0.91      8891




In [22]:
# For user input 
n=colnames.shape[0]
test_input=[]
for i in range(n):
    print("Do you have ", colnames[i], "?? Enter 1 for Yes and 0 for No-")
    inp=input()
    test_input.append(inp)
test_input=np.asarray(test_input)
test_input = test_input.reshape(1, -1)  #reshaping because right now shape of array is (n,) which has to be converted to (1,n)
test_input = scaler.transform(test_input)
key_list = list(dictionary.keys()) #make a list of keys 
val_list = list(dictionary.values()) #make a list of values 

# load the Neural Network model from disk
nn_model = keras.models.load_model("Symptoms_Keras_Model.h5")

prediction1=nn_model(test_input) 
prediction1=np.asarray(prediction1)
max_index_col = np.argmax(prediction1, axis=1) #find the max value in the output vector
print("NN says you have", key_list[val_list.index(max_index_col)]) #printing the key value wrt the output given by NN


Do you have  COUGH ?? Enter 1 for Yes and 0 for No-
1
Do you have  MUSCLE_ACHES ?? Enter 1 for Yes and 0 for No-
1
Do you have  TIREDNESS ?? Enter 1 for Yes and 0 for No-
1
Do you have  SORE_THROAT ?? Enter 1 for Yes and 0 for No-
1
Do you have  RUNNY_NOSE ?? Enter 1 for Yes and 0 for No-
0
Do you have  STUFFY_NOSE ?? Enter 1 for Yes and 0 for No-
0
Do you have  FEVER ?? Enter 1 for Yes and 0 for No-
1
Do you have  NAUSEA ?? Enter 1 for Yes and 0 for No-
0
Do you have  VOMITING ?? Enter 1 for Yes and 0 for No-
0
Do you have  DIARRHEA ?? Enter 1 for Yes and 0 for No-
0
Do you have  SHORTNESS_OF_BREATH ?? Enter 1 for Yes and 0 for No-
1
Do you have  DIFFICULTY_BREATHING ?? Enter 1 for Yes and 0 for No-
0
Do you have  LOSS_OF_TASTE ?? Enter 1 for Yes and 0 for No-
1
Do you have  LOSS_OF_SMELL ?? Enter 1 for Yes and 0 for No-
1
Do you have  ITCHY_NOSE ?? Enter 1 for Yes and 0 for No-
0
Do you have  ITCHY_EYES ?? Enter 1 for Yes and 0 for No-
0
Do you have  ITCHY_MOUTH ?? Enter 1 for Yes an

C:\Users\kimyo\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


NN says you have COLD
